# Catchment Analysis: Raster Preprocessing and MosaickingThis notebook performs the following steps:1. Load and inspect DEM TIFF files.2. Correct any upside-down (south-up) rasters to north-up orientation.3. Merge all TIFFs into one raster in chunks.4. Display the final merged raster.5. Clean up intermediate files.

## Chapter 1: Initialization and Data Loading

In [ ]:
import osimport rasteriofrom rasterio.merge import mergeimport matplotlib.pyplot as pltimport shutilimport numpy as npfrom affine import Affine# Set workspaceworkspace = r'C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis'folder_path = os.path.join(workspace, 'data')output_folder = os.path.join(workspace, 'north_up_rasters')os.makedirs(output_folder, exist_ok=True)# Get list of TIFF filestiff_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.tif') or f.lower().endswith('.tiff')]tiff_files.sort()joined_paths = sorted([os.path.join(folder_path, f) for f in tiff_files])# Open all images for inspection (optional)images = [rasterio.open(f) for f in joined_paths]

## Chapter 2: Correct upside-down (south-up) TIFFs to north-up

In [ ]:
for filename in os.listdir(folder_path):    if not filename.lower().endswith(".tif"):        continue    input_path = os.path.join(folder_path, filename)    output_path = os.path.join(output_folder, filename)    with rasterio.open(input_path) as src:        data = src.read(1)        transform = src.transform        profile = src.profile.copy()        if transform.e > 0:            print(f"Fixing upside-down raster: {filename}")            data = np.flipud(data)            new_transform = Affine(                transform.a, transform.b, transform.c,                transform.d, -transform.e, transform.f + (data.shape[0] * transform.e)            )            profile.update(transform=new_transform)        else:            new_transform = transform        with rasterio.open(output_path, "w", **profile) as dst:            dst.write(data, 1)joined_paths_northUp = sorted([os.path.join(output_folder, f) for f in tiff_files])

## Chapter 3: Merge TIFFs into one mosaic (chunked merging)

In [ ]:
chunk_size = 30chunks = [joined_paths_northUp[i:i + chunk_size] for i in range(0, len(joined_paths_northUp), chunk_size)]intermediate_files = []intermediate_folder = os.path.join(workspace, 'intermediate_results')os.makedirs(intermediate_folder, exist_ok=True)for i, chunk in enumerate(chunks):    srcs = [rasterio.open(f) for f in chunk]    mosaic, transform = merge(srcs)    out_meta = srcs[0].meta.copy()    out_meta.update({        "driver": "GTiff",        "height": mosaic.shape[1],        "width": mosaic.shape[2],        "transform": transform    })    out_path = os.path.join(intermediate_folder, f"intermediate_merge_{i}.tif")    if os.path.exists(out_path):        os.remove(out_path)    with rasterio.open(out_path, "w", **out_meta) as dest:        dest.write(mosaic)    intermediate_files.append(out_path)    for src in srcs:        src.close()print("Intermediate files written.")

## Step 4: Merge intermediate files into final output

In [ ]:
srcs = [rasterio.open(f) for f in intermediate_files]final_mosaic, final_transform = merge(srcs)out_meta = srcs[0].meta.copy()out_meta.update({    "driver": "GTiff",    "height": final_mosaic.shape[1],    "width": final_mosaic.shape[2],    "transform": final_transform})final_output_path = os.path.join(workspace, "final_merged_output.tif")if os.path.exists(final_output_path):    os.remove(final_output_path)with rasterio.open(final_output_path, "w", **out_meta) as dest:    dest.write(final_mosaic)for src in srcs:    src.close()print("Final merge complete:", final_output_path)

## Step 5: Display the final merged raster

In [ ]:
with rasterio.open(final_output_path) as finalMerged:    img = finalMerged.read(1)    plt.figure(figsize=(10, 10))    plt.imshow(img, cmap='terrain')    plt.axis('off')    plt.title("Final Merged Raster")    plt.show()

## Step 6: Clean up intermediate files

In [ ]:
if os.path.exists(intermediate_folder):    shutil.rmtree(intermediate_folder)    print("Intermediate files removed.")